In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import glob
from pathlib import Path

# 현재 디렉토리 (Untitled.ipynb 위치)
base_dir = "."

print("="*100)
print("🚀 Fold 1~8 전체 분석 시작")
print("="*100)

# === 1. Fold별 데이터 수집 ===
all_fold_results = []
all_predictions = {}  # 모델별로 전체 prediction 저장

for fold_idx in range(1, 9):
    if fold_idx < 8:
        fold_name = f'fold_{fold_idx}_walk_forward_rolling_reverse'
    else:
        fold_name = 'fold_8_final_holdout'
    
    fold_path = Path(base_dir) / fold_name
    
    print(f"\n{'='*80}")
    print(f"📂 Loading {fold_name}")
    print(f"{'='*80}")
    
    # 모든 모델의 prediction 파일
    pred_files = glob.glob(f"{fold_path}/*_predictions.csv")
    
    if len(pred_files) == 0:
        print(f"⚠️ No prediction files found in {fold_name}")
        continue
    
    print(f"Found {len(pred_files)} models")
    
    for pred_file in pred_files:
        model_name = Path(pred_file).stem.replace('_predictions', '')
        
        df = pd.read_csv(pred_file)
        df['fold'] = fold_idx
        
        # 기본 통계
        cm = confusion_matrix(df['actual_direction'], df['pred_direction'], labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()
        
        up_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        up_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        up_f1 = 2 * (up_precision * up_recall) / (up_precision + up_recall) if (up_precision + up_recall) > 0 else 0
        
        num_trades = (df['pred_direction'] == 1).sum()
        win_rate = tp / num_trades if num_trades > 0 else 0
        
        all_fold_results.append({
            'Fold': fold_idx,
            'Model': model_name,
            'Samples': len(df),
            'Up_Recall': up_recall,
            'Up_Precision': up_precision,
            'Up_F1': up_f1,
            'Win_Rate': win_rate,
            'Num_Trades': num_trades,
            'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn
        })
        
        # 전체 예측 저장 (fold 8 제외 - final holdout은 별도)
        if fold_idx < 8:
            if model_name not in all_predictions:
                all_predictions[model_name] = []
            all_predictions[model_name].append(df)

print(f"\n✅ 총 {len(all_fold_results)}개 결과 수집 완료")

# DataFrame으로 변환
fold_results_df = pd.DataFrame(all_fold_results)

# === 2. 모델별 평균 성능 (Fold 1~7만) ===
print("\n" + "="*100)
print("📊 모델별 평균 성능 (Fold 1~7, Walk-Forward)")
print("="*100)

fold17_results = fold_results_df[fold_results_df['Fold'] < 8]

model_avg = fold17_results.groupby('Model').agg({
    'Up_Recall': ['mean', 'std', 'min', 'max'],
    'Up_Precision': ['mean', 'std'],
    'Up_F1': ['mean', 'std'],
    'Win_Rate': ['mean', 'std'],
    'Num_Trades': 'sum'
}).round(4)

model_avg.columns = ['_'.join(col).strip() for col in model_avg.columns.values]
model_avg = model_avg.sort_values('Up_F1_mean', ascending=False)

print(model_avg.to_string())

# === 3. 전체 기간 통합 평가 (Fold 1~7) ===
print("\n" + "="*100)
print("🔥 전체 기간 통합 평가 (Fold 1~7 Test 합산)")
print("="*100)

combined_results = []

for model_name, dfs in all_predictions.items():
    # 모든 fold 합치기 (시간순)
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df = combined_df.sort_values('date')
    
    # 전체 통계
    total_samples = len(combined_df)
    
    cm = confusion_matrix(combined_df['actual_direction'], combined_df['pred_direction'], labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()
    
    up_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    up_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    up_f1 = 2 * (up_precision * up_recall) / (up_precision + up_recall) if (up_precision + up_recall) > 0 else 0
    
    num_trades = (combined_df['pred_direction'] == 1).sum()
    win_rate = tp / num_trades if num_trades > 0 else 0
    
    combined_results.append({
        'Model': model_name,
        'Total_Samples': total_samples,
        'Up_Recall': up_recall,
        'Up_Precision': up_precision,
        'Up_F1': up_f1,
        'Win_Rate': win_rate,
        'Num_Trades': num_trades,
        'Wins': tp,
        'Losses': num_trades - tp
    })

combined_df_results = pd.DataFrame(combined_results)
combined_df_results = combined_df_results.sort_values('Up_F1', ascending=False)

print("\n📈 전체 기간 성능 순위:")
print(combined_df_results.to_string(index=False))

# === 4. Fold 8 (Final Holdout) 성능 ===
print("\n" + "="*100)
print("🎯 Fold 8 (Final Holdout) 성능")
print("="*100)

fold8_results = fold_results_df[fold_results_df['Fold'] == 8].sort_values('Up_F1', ascending=False)
print(fold8_results[['Model', 'Up_F1', 'Up_Recall', 'Up_Precision', 'Win_Rate', 'Num_Trades']].to_string(index=False))

# === 5. 일관성 분석 ===
print("\n" + "="*100)
print("📈 모델별 일관성 분석 (Fold 1~7)")
print("="*100)

consistency = fold17_results.groupby('Model')['Up_F1'].agg(['mean', 'std', 'min', 'max'])
consistency['cv'] = consistency['std'] / consistency['mean']  # 변동계수 (낮을수록 일관적)
consistency = consistency.sort_values('mean', ascending=False)

print(consistency.to_string())

# === 6. 최종 추천 ===
print("\n" + "="*100)
print("🏆 최종 추천 모델")
print("="*100)

# 기준 설정
print("\n기준:")
print("  - 전체 Up_F1 > 0.70")
print("  - 전체 Up_Recall > 0.70")
print("  - 전체 Win_Rate > 0.60")
print("  - Fold8 Up_F1 > 0.70")

# 필터링
qualified = combined_df_results[
    (combined_df_results['Up_F1'] > 0.70) & 
    (combined_df_results['Up_Recall'] > 0.70) & 
    (combined_df_results['Win_Rate'] > 0.60)
]

if len(qualified) > 0:
    for idx, row in qualified.iterrows():
        model_name = row['Model']
        
        # Fold 8 성능 확인
        fold8_perf = fold8_results[fold8_results['Model'] == model_name]
        
        if len(fold8_perf) > 0 and fold8_perf.iloc[0]['Up_F1'] > 0.70:
            print(f"\n✅ {model_name}")
            print(f"   [Fold 1~7 통합]")
            print(f"     Up_F1: {row['Up_F1']:.4f}")
            print(f"     Up_Recall: {row['Up_Recall']:.2%}")
            print(f"     Win_Rate: {row['Win_Rate']:.2%}")
            print(f"     총 거래: {row['Num_Trades']:.0f}회 (승: {row['Wins']}, 패: {row['Losses']})")
            
            print(f"   [Fold 8 Holdout]")
            f8 = fold8_perf.iloc[0]
            print(f"     Up_F1: {f8['Up_F1']:.4f}")
            print(f"     Up_Recall: {f8['Up_Recall']:.2%}")
            print(f"     Win_Rate: {f8['Win_Rate']:.2%}")
            print(f"     거래: {f8['Num_Trades']:.0f}회")
            
            # 일관성
            cons = consistency.loc[model_name]
            print(f"   [일관성]")
            print(f"     F1 평균: {cons['mean']:.4f} ± {cons['std']:.4f}")
            print(f"     변동계수: {cons['cv']:.4f} (낮을수록 안정적)")
            print(f"     F1 범위: [{cons['min']:.4f}, {cons['max']:.4f}]")
            
            break
    else:
        print("\n⚠️ 모든 기준을 만족하는 모델 없음 (Fold8 포함)")
else:
    print("\n⚠️ 기준을 만족하는 모델 없음")
    print("\n차선책 (Top 3):")
    for idx, row in combined_df_results.head(3).iterrows():
        print(f"\n{idx+1}. {row['Model']}")
        print(f"   F1: {row['Up_F1']:.4f}, Recall: {row['Up_Recall']:.2%}, Win Rate: {row['Win_Rate']:.2%}")




🚀 Fold 1~8 전체 분석 시작

📂 Loading fold_1_walk_forward_rolling_reverse
Found 12 models

📂 Loading fold_2_walk_forward_rolling_reverse
Found 12 models

📂 Loading fold_3_walk_forward_rolling_reverse
Found 12 models

📂 Loading fold_4_walk_forward_rolling_reverse
Found 12 models

📂 Loading fold_5_walk_forward_rolling_reverse
Found 12 models

📂 Loading fold_6_walk_forward_rolling_reverse
Found 12 models

📂 Loading fold_7_walk_forward_rolling_reverse
Found 12 models

📂 Loading fold_8_final_holdout
Found 12 models

✅ 총 96개 결과 수집 완료

📊 모델별 평균 성능 (Fold 1~7, Walk-Forward)
                      Up_Recall_mean  Up_Recall_std  Up_Recall_min  Up_Recall_max  Up_Precision_mean  Up_Precision_std  Up_F1_mean  Up_F1_std  Win_Rate_mean  Win_Rate_std  Num_Trades_sum
Model                                                                                                                                                                                     
CatBoost                      0.7866         0.1244         0